In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# carregamento de bibliotecas
import sys
import os

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.ingestion import cotacao_atual_etanol_html
from src.features.data import criar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas, data_ingestao

In [0]:
# captura da cotação atual
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.precos_etanol_incremental_raw"
URL = "https://www.noticiasagricolas.com.br/cotacoes/sucroenergetico/indicador-semanal-etanol-hidratado-outros-fins-cepea-esalq"

df_cotacao = cotacao_atual_etanol_html(URL)

df_cotacao.head()

In [0]:
# sanitização das colunas
colunas_sanitizacao = {
    "R$/Litro":"Reais_por_litro", 
    "Variação/Semana (%)": "Variacao_Semana_pct"}


df_limpo = renomear_colunas(df=df_cotacao, 
                            dicionario= colunas_sanitizacao)
df_limpo.head()

In [0]:
# inserção da coluna de data de ingestao dos dados

df_limpo = data_ingestao(df=df_limpo)
df_limpo.head()

In [0]:
# transformação do dataset em dataframe spark
df_spark = criar_tabela_spark(df_limpo)

In [0]:
# persistência da tabela na camada bronze
salvar_tabela_delta_spark(df_spark,mode='append', partitionby=['Data'],merge_schema=True, path=TABELA_INCREMENTAL_RAW)


In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.precos_etanol_incremental_raw
LIMIT 10